In [4]:
!pip install arxiv

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
%pip install wikipedia


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
# wiki = WikipediaQueryRun(api_wrapper=api_wrapper)   -------> Incorrect Wikipedia wrapper

In [ ]:
# wiki.name = "wikipedia"
# wiki.description = "Search Wikipedia for general information."    ----> No need 

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "cpu"}
)


/tmp/ipykernel_5510/2877086701.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/adwait/anaconda3/envs/rag-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x71b257eb1150>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool
#retriever_tool = create_retriever_tool(retriever,"langsmith_search","search for information about langsmith. For any questions about langsmith you must use this tool")

In [32]:
retriever_tool = create_retriever_tool(
    retriever,
    name="Langsmith",
    description="Use this tool to search Langsmith documentation."
)

In [33]:
retriever_tool.name

'Langsmith'

In [34]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [35]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(),
    name="wikipedia",
    description="Use this tool to search for factual information from Wikipedia."
)



In [23]:
tools  = [wiki,arxiv,retriever_tool]

In [24]:
##Agents - core idea is to use a llm to choose a sequence of steps
from langchain.agents import initialize_agent,AgentType
from langchain.llms.ollama import Ollama

llm = Ollama(model = "llama3.2")

In [25]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True 
)

In [33]:
"""pip install langchainhub
"""

'pip install langchainhub\n'

In [31]:
"""from langchain import hub

prompt  = hub.pull("hwchase17/openai-functions-agent")
prompt.messages"""

'from langchain import hub\n\nprompt  = hub.pull("hwchase17/openai-functions-agent")\nprompt.messages'

In [17]:
"""##agent executer 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor""" """only needed when not initialized using initialize agent  """

'##agent executer \nfrom langchain.agents import AgentExecutor\nagent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)\nagent_executoronly needed when not initialized using initialize agent  '

In [26]:
from langchain.agents.agent import AgentExecutor
from langchain.agents import create_react_agent
from langchain.prompts import ChatPromptTemplate

prompt_template = """
Answer the following question as best you can. You can use the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original question

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [27]:
response = agent.invoke({"input":"who discovered gravity","intermediate_steps":[]})

In [28]:
print(response)


tool='wikipedia' tool_input='gravity' log='Thought: I need to find information about gravity to answer this question.\nAction: wikipedia\nAction Input: "gravity"'


In [36]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "what is langsmith"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To find information about Langsmith, I should use the langsmith_search tool.
Action: langsmith_search
Action Input: query='langsmith'Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your

KeyboardInterrupt: 